## 5.6　Affine／Softmax レイヤの実装

In [1]:
import numpy as np

### 5.6.1　Affine レイヤ

ニューラルネットワークの順伝播での行列の積は、幾何学の分野では「アフィン変換」と呼ばれる。<br>
アフィン変換を行う「Affine レイヤ」を実装していく。<br>
Affine レイヤとは、多次元の配列の計算に対応した $Y = X \cdot W + B$ の順伝播と逆伝播を行えるレイヤのことである。

In [2]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
dB = np.sum(dY, axis=0)

print(dY)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


$\frac{\partial L}{\partial Y}$ が $(N, 3)$　なのに対して $\frac{\partial L}{\partial B}$ は $(1, 3)$ であるから、$\frac{\partial L}{\partial Y}$ の要素を集約させる必要がある。<br>
そのために、最初の軸（第 0 軸）`axis=0` で和をとっている。

In [3]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

Affine レイヤとは、図5-27 の計算を行うレイヤである。

### 5.6.3　Softmax-with-Loss レイヤ

正規化とは、出力の和が 1 になるように変形することである。

図 5-30 を実装する。

In [4]:
class SoftmaxWithLoss:
    def __init__(self):
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx